In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import sympy as sp
import math
# !pip3 install git+https://github.com/Fangyh09/pytorch-receptive-field.git
from torch_receptive_field import receptive_field
from torchinfo import summary

import models as cifar_models

In [2]:
model = cifar_models.NetBNv2()

counter = 0
# Loop through each layer of the model
c_in, h_in, w_in = 3, 32, 32
for name, layer in model.named_children():
    # print(f"- Layer details: {name} | k={layer.kernel_size} | s={layer.stride} | p={layer.padding}")
    print(f"- Layer: {name}")
    if isinstance(layer, nn.Conv2d):
        out_channels = layer.out_channels
        k, s, p = layer.kernel_size[0], layer.stride[0], layer.padding[0]
        c_o, h_o, w_o = out_channels, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}")
        c_in, h_in, w_in = c_o, h_o, w_o
    
    elif isinstance(layer, nn.MaxPool2d):
        k, s, p = layer.kernel_size, layer.stride, layer.padding
        c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
        c_in, h_in, w_in = c_o, h_o, w_o

    elif isinstance(layer, nn.AvgPool2d):
        k, s, p = layer.kernel_size, layer.stride, layer.padding
        c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
        print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
        c_in, h_in, w_in = c_o, h_o, w_o

    elif isinstance(layer, nn.Sequential):
        for sub_name, sub_layer in layer.named_children():
            # print(f"- SubLayer details: {sub_name} | k={sub_layer.kernel_size} | s={sub_layer.stride} | p={sub_layer.padding}")
            if isinstance(sub_layer, nn.ReLU):
                continue
            print(f"- SubLayer: {sub_layer}")
            if isinstance(sub_layer, nn.Conv2d):
                out_channels = sub_layer.out_channels
                k, s, p = sub_layer.kernel_size[0], sub_layer.stride[0], sub_layer.padding[0]
                c_o, h_o, w_o = out_channels, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
                print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}")
                c_in, h_in, w_in = c_o, h_o, w_o
            
            elif isinstance(sub_layer, nn.MaxPool2d):
                k, s, p = sub_layer.kernel_size, sub_layer.stride, sub_layer.padding
                c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
                print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
                c_in, h_in, w_in = c_o, h_o, w_o

            elif isinstance(sub_layer, nn.AvgPool2d):
                k, s, p = sub_layer.kernel_size, sub_layer.stride, sub_layer.padding
                c_o, h_o, w_o = c_in, math.floor((h_in + 2*p - k)/s + 1), math.floor((w_in + 2*p - k)/s + 1)
                print(f"-------------------------------- no. of channels: {c_o} | feature map dim: {h_o}x{w_o}\n")
                c_in, h_in, w_in = c_o, h_o, w_o
    else:
        print('layer not implemented')
        break


- Layer: convBlock1
- SubLayer: Conv2d(3, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
-------------------------------- no. of channels: 32 | feature map dim: 32x32
- SubLayer: BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
- SubLayer: Dropout(p=0.1, inplace=False)
- Layer: convBlock2
- SubLayer: Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
-------------------------------- no. of channels: 32 | feature map dim: 32x32
- SubLayer: BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
- SubLayer: Dropout(p=0.1, inplace=False)
- Layer: convBlock3
- SubLayer: Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
-------------------------------- no. of channels: 16 | feature map dim: 32x32
- Layer: pool1
-------------------------------- no. of channels: 16 | feature map dim: 16x16

- Layer: convBlock4
- SubLayer: Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
-----------------------------

In [3]:
# receptive_field(model, input_size=(channels, H, W))
receptive_field(model, input_size=(3, 32, 32))

ValueError: module Dropout not ok

In [ ]:
model.summary(input_size=(1, 3, 32, 32))